In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath('../src'))

In [2]:
import numpy as np
from tqdm.notebook import trange
from tqdm.notebook import tqdm
from docplex.mp.model import Model

import model
import model_parameters as MP

## Mathematical Model

#### Sets
- C	Set of campaigns.
- U	Set of customers.
- H	Set of channels
- D	Set of planning days.
- I	Set of quota categories.
- P	Set of priority categories.


In [3]:
print(f"number of campaigns {MP.C}")
print(f"number of customers {MP.U}")
print(f"number of channels {MP.H}")
print(f"number of planning days {MP.D}")
print(f"number of quota categories {MP.I}")
print(f"number of priority categories {MP.P}")

number of campaigns 10
number of customers 2000
number of channels 3
number of planning days 7
number of quota categories 3
number of priority categories 10


#### Parameters

##### - eligibility
$$
e_{cu}\left\{\begin{array}\\
        1 & \mbox{if }  customer\ u\ is\ eligible\ for\ campaign\ c\\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [4]:
MP.e_cu

array([[1, 0, 0, ..., 1, 1, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 1, ..., 0, 1, 0],
       [1, 1, 1, ..., 0, 0, 1],
       [0, 1, 0, ..., 0, 1, 1]])

##### - quota categories
$$
q_{ic}\left\{\begin{array}\\
        1 & \mbox{if }  campaign\ c\ is\ a\ i^{th} type\ quota\ category\ campaign\ \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [5]:
MP.q_ic

array([[1, 0, 1, 1, 0, 0, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 0, 1, 1],
       [1, 1, 0, 1, 0, 0, 0, 1, 0, 0]])

##### - priority categories
$$r_{cp}: Priority\ value\ of\ campaign\ c\ regarding\ priority\ type\ p\$$

In [6]:
MP.rp_c

array([ 8,  8,  7,  8,  7,  8, 44, 66,  7, 74])

##### - blokage
$$b: Communication\ limit\ per\ person\ for\ the\ whole\ period\$$

In [7]:
MP.b

7

##### - daily blokage
$$k: Communication\ limit\ per\ person\ at\ each\ day\$$

In [8]:
MP.k

3

##### - campaign blockage
$$l_c: Communication\ limit\ per\ person\ for\ campaign\ c\$$

In [9]:
MP.l_c

array([4, 3, 2, 2, 2, 2, 4, 4, 2, 2])

##### - quota limitations daily/weekly
$$
m_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\
$$
$$
n_i: Communication\ limit\ per\ person\ for\ i^{th}\ category\ each\ day\
$$

In [10]:
(MP.m_i, MP.n_i)

(array([3, 5, 4]), array([2, 2, 3]))

#### - capacity for channel
$$
t_{h,d}: Capacity\ for\ channel\ h\ at\ day\ d.\
$$

In [11]:
MP.t_hd

array([[1000., 1200., 1400., 1200., 1200., 1400., 1400.],
       [1200., 1000., 1200., 1200., 1400., 1400., 1200.],
       [1000., 1000., 1400., 1000., 1200., 1200., 1000.]])

### Model

In [12]:
mdl = Model(name='Campaign Optimization')

#### Variables
$$
X_{cuhd}\left\{\begin{array}\\
        1 & \mbox{if } Campaign\ c\ will\ be\ sent\ to\ customer\ u\ through\ Channel\ h\ at\ Day\ d \\
        0 & \mbox{otherwise } \\
    \end{array}
\right.
$$

In [13]:
X_cuhd = {(c,u,h,d): mdl.continuous_var(lb=0, ub=1, name=f"X_c:{c}_u:{u}_h:{h}_d:{d}")
            for c in range(0,MP.C)
            for u in range(0,MP.U) 
            for h in range(0,MP.H)
            for d in range(0,MP.D)}

## Maximize
$$\sum_{p \in P}\sum_{c \in C}\sum_{u \in U}\sum_{h \in H}\sum_{d \in D}\,X_{cuhd}\ r_{cp}$$

##### Binary variable (10)
$$
X_{cuhd} \in \{ 1,0 \},\hspace{35pt} \forall c \in C ,\forall u \in U,\forall d \in D, \forall h \in H
$$

In [14]:
maximize = mdl.maximize(mdl.sum([X_cuhd[(c,u,h,d)] * MP.rp_c[c]
                  for c in range(0,MP.C)
                  for u in range(0,MP.U) 
                  for h in range(0,MP.H) 
                  for d in range(0,MP.D)]))

## subject to

##### - eligibility (2)

$$
X_{cuhd}  \leq e_{cu},\hspace{35pt} \forall c \in C,\forall u \in U,\forall h \in H,\forall d \in D
$$

In [15]:
eligibilitiy = mdl.add_constraints(
    (X_cuhd[(c,u,h,d)] <= MP.e_cu[c,u]
    for c in range(0,MP.C)
    for u in range(0,MP.U) 
    for h in range(0,MP.H) 
    for d in range(0,MP.D))
)

##### - use one channel (3)
$$
\sum_{h}X_{cuhd} \le 1,\hspace{35pt} \forall c \in C \, \forall u \in U,\forall d \in D
$$

In [16]:
one_channel = mdl.add_constraints(
    ((mdl.sum(X_cuhd[(c,u,h,d)] for h in range(0,MP.H)) <= 1)
    for c in range(0,MP.C)
    for u in range(0,MP.U) 
    for d in range(0,MP.D))
)

##### - weekly communication limitation (4)
$$
\sum_{h \in H}\sum_{c \in C}\sum_{d \in D} X_{cuhd}\le b,\hspace{35pt} \forall u \in U
$$

In [17]:
weekly_communication = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)] 
               for d in range(0,MP.D) 
               for c in range(0,MP.C) 
               for h in range(0,MP.H)) <= MP.b)
        for u in range(0,MP.U)))

##### - daily communication limitation (5)
$$
\sum_{h \in H}\sum_{c \in C} X_{cuhd}\le k,\hspace{35pt} \forall u \in U, \forall d \in D
$$

In [18]:
daily_communication = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]  
               for c in range(0,MP.C) 
               for h in range(0,MP.H)) <= MP.k)
        for d in range(0,MP.D)
        for u in range(0,MP.U)))

##### - campaign communication limit(6)
$$
\sum_{d \in D}\sum_{h \in H} X_{cuhd}\le l_c,\hspace{35pt} \forall c \in C,\forall u \in U;
$$

In [19]:
campaign_communication = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]  
               for h in range(0,MP.H) 
               for d in range(0,MP.D)) <= MP.l_c[c] )
        for c in range(0,MP.C)
        for u in range(0,MP.U)))

##### - weekly quota(7)
$$
\sum_{d \in D}\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le m_i,\hspace{35pt} \forall u \in U, \forall i \in I
$$

In [20]:
weekly_quota = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]*MP.q_ic[i,c]
               for c in range(0,MP.C)
               for h in range(0,MP.H) 
               for d in range(0,MP.D)) <= MP.m_i[i])
        for u in range(0,MP.U)
        for i in range(0,MP.I)))

##### - daily quota(8)
$$
\sum_{h \in H}\sum_{c \in C}{X_{cuhd} q_{ic}}\le n_i,\hspace{35pt} \forall u \in U,\, \forall d \in D, \forall i \in I
$$

In [21]:
daily_quota = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]*MP.q_ic[i,c]
               for c in range(0,MP.C) 
               for h in range(0,MP.H)) <= MP.n_i[i])
        for u in range(0,MP.U)
        for d in range(0,MP.D)
        for i in range(0,MP.I)))

##### Channel capacity (9)
$$
\sum_{c \in C}\sum_{u \in U}{X_{cuhd}}\le t_{hd},\hspace{35pt} \forall d \in D,\, \forall h \in H
$$

TODO: kampanya kanal.

In [22]:
channel_capacity = mdl.add_constraints(
    (
        (mdl.sum(X_cuhd[(c,u,h,d)]
               for u in range(0,MP.U) 
               for c in range(0,MP.C)) <= MP.t_hd[h,d])
        for h in range(0,MP.H)
        for d in range(0,MP.D)))

### Solution

In [23]:
%%time

solution = mdl.solve(log_output=True)
#solution = mdl.solve(log_output=True)

Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              202001241
Parallel mode: deterministic, using up to 4 threads for concurrent optimization:
 * Starting dual Simplex on 1 thread...
 * Starting Barrier on 3 threads...
Tried aggregator 1 time.
LP Presolve eliminated 512120 rows and 208341 columns.
Reduced LP has 131901 rows, 211659 columns, and 1709169 nonzeros.
Presolve time = 1.89 sec. (731.28 ticks)
Symmetry aggregator did 276494 additional substitutions.

Iteration log . . .
Iteration:     1   Dual objective     =       4341796.000000
Iteration:   432   Dual objective     =       1357334.000000
Iteration:   916   Dual objective     =        719702.000000
Iteration:  1452   Dual objective     =        587540.000000
Iteration:  2104   Dual objective     =        551588.000000
Perturbation started.
Iteration:  2201   Dual objective     =        551588.000000
Iteration:  2751   Dual ob

In [24]:
solution.objective_value

366401.0

In [25]:
solution.as_df().sum()["value"]

10367.0

In [26]:
display(len([val for ky,val in solution.as_name_dict().items() if val<0.5]))
display(len([val for ky,val in solution.as_name_dict().items() if val>0.5]))
display(len([val for ky,val in solution.as_name_dict().items() if val<1]))
display(len([val for ky,val in solution.as_name_dict().items() if val==0.5]))
display(len([val for ky,val in solution.as_name_dict().items() if val==1]))

0

10357

20

20

10357

In [27]:
X_non_integral = [tuple(int(i.split(":")[1]) for i in ky.split("_")[1:]) for ky,val in solution.as_name_dict().items() if val!=1]
X_cuhd2 = np.zeros((MP.C,MP.U,MP.H,MP.D), dtype='int')
for ky,_ in solution.as_name_dict().items():
    exec(f'X_cuhd2{[int(i.split(":")[1]) for i in ky.split("_")[1:]]} = 1', {}, {'X_cuhd2':X_cuhd2})

c_i = 0
u_i = 1
h_i = 2
d_i = 3

mdl = model.Model([
    model.Constraint('eligibility',MP.eligibility, (c_i, u_i, h_i, d_i,)),
    model.Constraint('channel_capacity',MP.channel_capacity, (h_i, d_i,)),
    model.Constraint('daily_limitation',MP.daily_limitation, (u_i, d_i,)),
    model.Constraint('weekly_limitation',MP.weekly_limitation, (u_i,)),
    model.Constraint('campaign_limitation',MP.campaign_limitation, (c_i, u_i,)),
    model.Constraint('daily_quota',MP.daily_quota, (u_i, d_i,)),
    model.Constraint('one_channel',MP.one_channel, (c_i, u_i, d_i,)),
    model.Constraint('weekly_quota',MP.weekly_quota, (u_i,))
], MP.objective_fn)

def round_with_greedy():
    for c in tqdm(np.argsort(MP.rp_c), desc="Campaigns Loop"):
        for d in trange(MP.D, desc=f"Days Loop for campaign-{c}"):
            for h in range(MP.H):
                for u in range(MP.U):
                    if (c,u,h,d) in X_non_integral and X_cuhd2[c,u,h,d]==1 and not mdl.execute(X_cuhd2, (c, u, h, d)):
                        X_cuhd2[c,u,h,d]=0

def validate():
    for c in tqdm(np.argsort(MP.rp_c), desc="Campaigns Loop"):
        for d in trange(MP.D, desc=f"Days Loop for campaign-{c}"):
            for h in range(MP.H):
                for u in range(MP.U):
                    if not mdl.execute(X_cuhd2, (c, u, h, d)):
                        print(f"X({(c,u,h,d)}) causes invalidation.")
print("Rounding")
round_with_greedy()
print("Validation")
validate()

model_value = mdl.calc_value(X_cuhd2)
model_comm_count = X_cuhd2.sum()

print(f"Solution is ({model_value})")
print(f"Solution communication count is ({model_comm_count})")

Rounding


Campaigns Loop:   0%|          | 0/10 [00:00<?, ?it/s]

Days Loop for campaign-2:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-4:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-8:   0%|          | 0/7 [00:00<?, ?it/s]

...

Days Loop for campaign-5:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-6:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-7:   0%|          | 0/7 [00:00<?, ?it/s]

Days Loop for campaign-9:   0%|          | 0/7 [00:00<?, ?it/s]

Solution is (366401)
Solution communication count is (10364)
